## Esercizio

Esercizio 3 dell'esame di settembre 2023. Dato il tipo di dato `colored_tree`, definire due funzioni
```
color_tree_lists : colored_tree -> int list * int list * int list
color_forest_lists : colored_tree list -> int list * int list * int list
```

`color_tree_lists` deve restiture tre liste, una con tutti i valori nei nodi neri, una con tutti i valori nei nodi rossi e una con tutti i valori nei nodi verdi.
`color_forest_lists` fa la stessa cosa, ma a partire di una lista di alberi: la prima lista contiene quindi tutti i valori nei nodi neri di tutti gli alberi nella lista di input, e così via.

In [1]:
type colored_tree =
    | BlackNode of int * colored_tree * colored_tree
    | RedNode of int * colored_tree * colored_tree
    | GreenLeaf of int;;

type colored_tree =
    BlackNode of int * colored_tree * colored_tree
  | RedNode of int * colored_tree * colored_tree
  | GreenLeaf of int


In [2]:
let rec color_tree_lists ctree =
  match ctree with
  | GreenLeaf n -> ([], [], [n])
  | BlackNode (n, ltree, rtree) ->
      let (lbnodes, lrnodes, lgnodes) = color_tree_lists ltree in
      let (rbnodes, rrnodes, rgnodes) = color_tree_lists rtree in
      ([n]@lbnodes@rbnodes, lrnodes@rrnodes, lgnodes@rgnodes)
  | RedNode (n, ltree, rtree) ->
      let (lbnodes, lrnodes, lgnodes) = color_tree_lists ltree in
      let (rbnodes, rrnodes, rgnodes) = color_tree_lists rtree in
      (lbnodes@rbnodes, [n]@lrnodes@rrnodes, lgnodes@rgnodes);;

let ct = BlackNode (10,
    RedNode (5,
        BlackNode (2,
            GreenLeaf 1,
            GreenLeaf 2),
        GreenLeaf 4),
    GreenLeaf 2);;
color_tree_lists ct;;

val color_tree_lists : colored_tree -> int list * int list * int list = <fun>


val ct : colored_tree =
  BlackNode (10,
   RedNode (5, BlackNode (2, GreenLeaf 1, GreenLeaf 2), GreenLeaf 4),
   GreenLeaf 2)


- : int list * int list * int list = ([10; 2], [5], [1; 2; 4; 2])


In [3]:
let color_forest_lists (ctrees : colored_tree list) =
  let rec aux (ctrees : colored_tree list) (partial_res : int list * int list * int list) =
    let (bns, rns, gns) = partial_res in
    match ctrees with
    | [] -> partial_res
    | ctree::rest -> let (bn, rn, gn) = color_tree_lists ctree in
      aux rest (bn@bns, rn@rns, gn@gns)
  in
  aux ctrees ([], [], []);;

color_forest_lists [ct; ct];;

val color_forest_lists : colored_tree list -> int list * int list * int list =
  <fun>


- : int list * int list * int list =
([10; 2; 10; 2], [5; 5], [1; 2; 4; 2; 1; 2; 4; 2])


In [4]:
List.fold_left;;

let color_forest_lists (ctrees : colored_tree list) =
  List.fold_left (fun partial_res ctree ->
      let (bns, rns, gns) = partial_res in
      let (bn, rn, gn) = color_tree_lists ctree in
        (bn@bns, rn@rns, gn@gns)
    ) ([], [], []) ctrees;;

color_forest_lists [ct; ct];;

let color_forest_lists (ctrees : colored_tree list) =
  let tmp = List.map color_tree_lists ctrees in
  List.fold_left (fun partial_res nodes_triple ->
      let (bns, rns, gns) = partial_res in
      let (bn, rn, gn) = nodes_triple in
        (bn@bns, rn@rns, gn@gns)
    ) ([], [], []) tmp;;

color_forest_lists [ct; ct];;

- : ('a -> 'b -> 'a) -> 'a -> 'b list -> 'a = <fun>


val color_forest_lists : colored_tree list -> int list * int list * int list =
  <fun>


- : int list * int list * int list =
([10; 2; 10; 2], [5; 5], [1; 2; 4; 2; 1; 2; 4; 2])


val color_forest_lists : colored_tree list -> int list * int list * int list =
  <fun>


- : int list * int list * int list =
([10; 2; 10; 2], [5; 5], [1; 2; 4; 2; 1; 2; 4; 2])


## Esercizio

Esercizio 4 dell'esame di settembre 2023. Estendere MiniCaml con collezioni di interi `IntCollection` e due operazioni `Union` e `Exists`.

Come tutti gli esercizi dell'interprete, procediamo in 4 passaggi.

### Espressioni

In [5]:
(* Identificatori *)
type ide = string;;

(* I tipi *)
type tname =  TInt | TBool | TString | TClosure | TRecClosure | TUnBound | TIntCollection

(* Abstract Expressions = espressioni nella sintassi astratta, 
   compongono l'Albero di Sintassi Astratta *)
type exp = 
    | EInt of int
    | CstTrue 
    | CstFalse
    | EString of string
    | Den of ide
    (* Operatori binari da interi a interi*)
    | Sum of exp * exp
    | Diff of exp * exp
    | Prod of exp * exp
    | Div of exp * exp
    (* Operatori da interi a booleani *)
    | IsZero of exp
    | Eq of exp * exp
    | LessThan of exp*exp
    | GreaterThan of exp*exp
    (* Operatori su booleani *)
    | And of exp*exp
    | Or of exp*exp
    | Not of exp
    (* Controllo del flusso, funzioni *)
    | IfThenElse of exp * exp * exp
    | Let of ide * exp * exp
    | Letrec of ide * ide  * exp * exp
    | Fun of ide * exp
    | Apply of exp * exp
    (* Costrutto Collection *)
    | Collection of exp list
    | Union of exp * exp
    | Exists of exp * exp
;;

type ide = string


type tname =
    TInt
  | TBool
  | TString
  | TClosure
  | TRecClosure
  | TUnBound
  | TIntCollection


type exp =
    EInt of int
  | CstTrue
  | CstFalse
  | EString of string
  | Den of ide
  | Sum of exp * exp
  | Diff of exp * exp
  | Prod of exp * exp
  | Div of exp * exp
  | IsZero of exp
  | Eq of exp * exp
  | LessThan of exp * exp
  | GreaterThan of exp * exp
  | And of exp * exp
  | Or of exp * exp
  | Not of exp
  | IfThenElse of exp * exp * exp
  | Let of ide * exp * exp
  | Letrec of ide * ide * exp * exp
  | Fun of ide * exp
  | Apply of exp * exp
  | Collection of exp list
  | Union of exp * exp
  | Exists of exp * exp


### Tipi esprimibili

In [6]:
(* Ambiente polimorfo *)
type 't env = ide -> 't

(* Evaluation types = tipi  esprimibili *)
type evT = 
    | Int of int 
    | Bool of bool 
    | String of string 
    | Closure of ide * exp * evT env 
    | RecClosure of ide * ide * exp * evT env
    | UnBound
    | IntCollection of int list

(* Ambiente vuoto *)
let emptyenv = function x -> UnBound

(* Binding fra una stringa x e un valore primitivo evT *)
let bind (s: evT env) (x: ide) (v: evT) = 
    function (i: ide) -> if (i = x) then v else (s i)

type 't env = ide -> 't


type evT =
    Int of int
  | Bool of bool
  | String of string
  | Closure of ide * exp * evT env
  | RecClosure of ide * ide * exp * evT env
  | UnBound
  | IntCollection of int list


val emptyenv : 'a -> evT = <fun>


val bind : evT env -> ide -> evT -> ide -> evT = <fun>


### Typechecking

In [7]:
(* Funzione da evT a tname che associa a ogni valore il suo descrittore di tipo  *)
let getType (x: evT) : tname =
    match x with
    | Int(n) -> TInt
    | Bool(b) -> TBool
    | String(s) -> TString
    | Closure(i,e,en) -> TClosure
    | RecClosure(i,j,e,en) -> TRecClosure
    | UnBound -> TUnBound
    | IntCollection(_) -> TIntCollection

(* Type-checking *)
let typecheck ((x, y) : (tname*evT)) = 
    match x with
    | TInt -> (match y with 
               | Int(u) -> true
               | _ -> false
               )
    | TBool -> (match y with 
                | Bool(u) -> true
                | _ -> false
                )
    | TString -> (match y with
                 | String(u) -> true
                 | _ -> false
                 )
    | TClosure -> (match y with
                   | Closure(i,e,n) -> true
                   | _ -> false
                   )
    | TRecClosure -> (match y with
                     | RecClosure(i,j,e,n) -> true
                     | _ -> false
                     )
    | TUnBound -> (match y with
                 | UnBound -> true
                 | _ -> false
                 )
    | TIntCollection -> (match y with
                 | IntCollection(_) -> true
                 | _ -> false
                 )

val getType : evT -> tname = <fun>


val typecheck : tname * evT -> bool = <fun>


### Interprete

In [8]:
(* Errori a runtime *)
exception RuntimeError of string

(* PRIMITIVE del linguaggio *)

(* Controlla se un numero è zero *)
let is_zero(x) = match (typecheck(TInt,x),x) with
    | (true, Int(v)) -> Bool(v = 0)
    | (_, _) -> raise ( RuntimeError "Wrong type")

(* Uguaglianza fra interi *)
let int_eq(x,y) =   
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v = w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Somma fra interi *)	   
let int_plus(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v + w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Differenza fra interi *)
let int_sub(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v - w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Prodotto fra interi *)
let int_times(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v * w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")
    
(* Divisione fra interi *)
let int_div(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> 
                    if w<>0 then Int(v / w)
                            else raise (RuntimeError "Division by zero")
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni di confronto *)
let less_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v < w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let greater_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v > w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni logiche *)
let bool_and(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v && w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_or(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v || w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_not(x) = 
    match (typecheck(TBool,x), x) with
    | (true, Bool(v)) -> Bool(not(v))
    | (_,_) -> raise ( RuntimeError "Wrong type")

exception RuntimeError of string


val is_zero : evT -> evT = <fun>


val int_eq : evT * evT -> evT = <fun>


val int_plus : evT * evT -> evT = <fun>


val int_sub : evT * evT -> evT = <fun>


...

val less_than : evT * evT -> evT = <fun>


val greater_than : evT * evT -> evT = <fun>


val bool_and : evT * evT -> evT = <fun>


val bool_or : evT * evT -> evT = <fun>


val bool_not : evT -> evT = <fun>


In [11]:
(* Interprete *)
let rec eval (e:exp) (s:evT env) : evT = 
    match e with
    | EInt(n) -> Int(n)
    | CstTrue -> Bool(true)
    | CstFalse -> Bool(false)
    | EString(s) -> String(s)
    | Den(i) -> (s i)

    | Prod(e1,e2) -> int_times((eval e1 s), (eval e2 s))
    | Sum(e1, e2) -> int_plus((eval e1 s), (eval e2 s))
    | Diff(e1, e2) -> int_sub((eval e1 s), (eval e2 s))
    | Div(e1, e2) -> int_div((eval e1 s), (eval e2 s))

    | IsZero(e1) -> is_zero (eval e1 s)
    | Eq(e1, e2) -> int_eq((eval e1 s), (eval e2 s))
    | LessThan(e1, e2) -> less_than((eval e1 s),(eval e2 s))
    | GreaterThan(e1, e2) -> greater_than((eval e1 s),(eval e2 s))

    | And(e1, e2) -> bool_and((eval e1 s),(eval e2 s))
    | Or(e1, e2) -> bool_or((eval e1 s),(eval e2 s))
    | Not(e1) -> bool_not(eval e1 s)

    | IfThenElse(e1,e2,e3) -> 
        let g = eval e1 s in 
            (match (typecheck(TBool,g),g) with
            |(true, Bool(true)) -> eval e2 s
            |(true, Bool(false)) -> eval e3 s
            |(_,_) -> raise ( RuntimeError "Wrong type")
            )
    | Let(i, e, ebody) -> eval ebody (bind s i (eval e s))
    | Fun(arg, ebody) -> Closure(arg,ebody,s)
    | Letrec(f, arg, fBody, leBody) ->
        let benv = bind (s) (f) (RecClosure(f, arg, fBody,s)) in
            eval leBody benv
    | Apply(eF, eArg) ->
        let fclosure = eval eF s in 
            (match fclosure with 
            | Closure(arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in 
                let aenv = bind fDecEnv arg aVal in 
                eval fbody aenv 
            | RecClosure(f, arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in
                let rEnv = bind fDecEnv f fclosure in
                let aenv = bind rEnv arg aVal in 
                eval fbody aenv
            | _ -> raise ( RuntimeError "Wrong type")
            )
    (* Nuovo costrutto IntCollection *)
    | Collection (elist) ->
        let val_list : evT list = List.map (fun e -> eval e s) elist in
        (* Collection ([Bool true]) *)
        if List.length val_list <> List.length (List.filter (fun v -> typecheck (TInt, v)) val_list) then
            raise ( RuntimeError "Wrong type")
        else
            let int_list = List.map (fun v -> match v with
                | Int n -> n
                | _ -> failwith "Impossibile"
            ) val_list
            in IntCollection int_list
    | Union(ec1, ec2) ->
        let c1 = eval ec1 s in
        let c2 = eval ec2 s in
        (* Union (Bool true, Bool false)*)
        (match (typecheck(TIntCollection,c1), typecheck(TIntCollection,c2), c1, c2) with
        | true, true, IntCollection(l1), IntCollection(l2) -> IntCollection(l1@l2)
        | _, _, _, _ -> raise ( RuntimeError "Wrong type")
        )
    | Exists(eP, eC) ->
        let p = eval eP s in
        let c = eval eC s in
        (match (typecheck(TClosure,p), typecheck(TIntCollection,c), p, c) with
        | true, true, _, IntCollection(l) ->
            (* Questa riga non funziona davvero perché usa valori OCaml quando
             * servirebbero dei valori MiniCaml e viceversa, però l'intuizione
             * è questa
             *)
            Bool (List.length (List.filter (fun v -> eval (Apply(p, v)) s) l) <> 0)
        | _ -> raise ( RuntimeError "Wrong type")
        )


error: compile_error